In [101]:
from pyspark.sql import SparkSession
from konlpy.tag import Mecab

In [102]:
spark = SparkSession.builder.getOrCreate()
mecab = Mecab()

In [103]:
df = spark.read.csv("/collected", sep=',', inferSchema=True, header=True)

In [104]:
df.printSchema()

root
 |-- text: string (nullable = true)
 |-- keyword: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- channel: string (nullable = true)



In [105]:
df.createOrReplaceTempView("COLLECTED")
text_df = spark.sql("SELECT text FROM COLLECTED")
text_df.printSchema()
text_df.show()

root
 |-- text: string (nullable = true)

+------------------------------------+
|                                text|
+------------------------------------+
|   인천 도림동에 위치한 오봉산칼국수|
|          만두전골 푸짐하게 먹고와서|
|                      포스팅해봅니다|
|                      외관사진입니다|
| 멀리서봐도 한눈에 보이는 외관입니다|
|   주차는 매장뒤쪽이나 매장 앞 도...|
|                        메뉴판입니다|
|      대표메뉴인 만두전골을 시작으로|
| 바지락칼국수 들깨칼국수 파전등이...|
|          만두전골2인 주문해봤습니다|
|           부추랑 양파가 들어간 소스|
|                        기본찬입니다|
|          배추김치 도토리묵 열무김치|
|  콘옥수수 마요네즈에 버무려서 살...|
|              고기가 먼저 나왔습니다|
|  다른 전골집에서 나오는 고기보다...|
|                   두께는 비슷했고요|
|           만두는 1인분에 3개씩 제공|
|         만두가 좀 적다고 생각했는데|
|버섯이 종류별로 굉장히 많이 나옵니다|
+------------------------------------+
only showing top 20 rows



In [106]:
from pyspark.ml.feature import Word2Vec

In [107]:
df = spark.read.csv("/collected", sep=',', inferSchema=True, header=True)

In [108]:
text_df = df.select("text").collect()

In [110]:
tuple_text_list = []
for row in text_df:
    text_morphs = mecab.morphs(row.text)
    proc_text_list = []
    for text in text_morphs:
        if len(text) > 1:
            proc_text_list.append(text)
    tuple_text_list.append((proc_text_list,))

docDF = spark.createDataFrame(tuple_text_list, ["text"])
print(docDF)

DataFrame[text: array<string>]


In [111]:
word2vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2vec.fit(docDF)
result = model.transform(docDF)

In [112]:
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

Text: [인천, 도림동, 위치, 오봉산, 칼국수] => 
Vector: [0.02717706188559532,-0.024426618590950966,0.029141636751592162]

Text: [만두전골, 푸짐, 와서] => 
Vector: [0.025229723503192265,-0.006144768248001734,-0.09346102736890316]

Text: [포스팅, 봅니다] => 
Vector: [0.029028380289673805,-0.025218552676960826,-0.047101955860853195]

Text: [외관, 사진, 입니다] => 
Vector: [-0.04719886680444081,0.0735671098033587,-0.03940579046805699]

Text: [멀리서, 봐도, 한눈, 보이, 외관, 입니다] => 
Vector: [-0.03586187089482943,0.023267429942886032,0.026415213321646053]

Text: [주차, 매장, 뒤쪽, 이나, 매장, 도로, 시간, 따라, 가능, 됩니다] => 
Vector: [0.05188747826032341,-0.11226623132824898,0.0028333596885204318]

Text: [메뉴판, 입니다] => 
Vector: [-0.05491422489285469,0.002169448882341385,-0.12054306268692017]

Text: [대표, 메뉴, 만두전골, 시작, 으로] => 
Vector: [-0.13300578743219377,0.06312273442745209,-0.038983010500669484]

Text: [바지락, 칼국수, 들깨, 칼국수, 파전, 습니다] => 
Vector: [0.009773743959764639,0.0325592216104269,0.12447478994727135]

Text: [만두전골, 주문, 습니다] => 
Vector: [-0.074492173890

In [69]:
rdd = sc.parallelize(text_df.collect())

In [70]:
rdd.collect()

[Row(text='인천 도림동에 위치한 오봉산칼국수'),
 Row(text='만두전골 푸짐하게 먹고와서'),
 Row(text='포스팅해봅니다'),
 Row(text='외관사진입니다'),
 Row(text='멀리서봐도 한눈에 보이는 외관입니다'),
 Row(text='주차는 매장뒤쪽이나 매장 앞 도로 시간에 따라 가능 에 하시면 됩니다'),
 Row(text='메뉴판입니다'),
 Row(text='대표메뉴인 만두전골을 시작으로'),
 Row(text='바지락칼국수 들깨칼국수 파전등이 있습니다'),
 Row(text='만두전골2인 주문해봤습니다'),
 Row(text='부추랑 양파가 들어간 소스'),
 Row(text='기본찬입니다'),
 Row(text='배추김치 도토리묵 열무김치'),
 Row(text='콘옥수수 마요네즈에 버무려서 살찔것같은데 먹게되는거'),
 Row(text='고기가 먼저 나왔습니다'),
 Row(text='다른 전골집에서 나오는 고기보다 큼직하게 나옵니다'),
 Row(text='두께는 비슷했고요'),
 Row(text='만두는 1인분에 3개씩 제공'),
 Row(text='만두가 좀 적다고 생각했는데'),
 Row(text='버섯이 종류별로 굉장히 많이 나옵니다'),
 Row(text='비주얼이 좋습니다'),
 Row(text='전골에 버섯넣으면 맛이 또 좋잖아요'),
 Row(text='팔팔 끓고 있습니다'),
 Row(text='고기도 조금씩 넣어서 이제 먹을준비 완료'),
 Row(text='버섯이랑 같이 먹어줍니다'),
 Row(text='고기도 괜찮고 버섯식감도 일품입니다'),
 Row(text='고기만두 속은 이런느낌입니다'),
 Row(text='김치만두 속은 이런느낌'),
 Row(text='만두는 김치만두가 더 맛있더라고요'),
 Row(text='고기추가 7000원'),
 Row(text='먹다보니 고기가 부족해서'),
 Row(text='먹다보니 고기만 남아서'),
 Row(text='육수까지 새로 부어서 먹는데'

In [ ]:
text_list = []

In [57]:
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
# model = word2Vec.fit(df2)

In [60]:
model = word2Vec.fit(df2.select("text"))

IllegalArgumentException: requirement failed: Column text must be of type equal to one of the following types: [array<string>, array<string>] but was actually of type string.

In [51]:
for row in df2.collect():
    
    print(mecab.morphs(row.text))

['인천', '도림동', '에', '위치', '한', '오봉산', '칼국수']
['만두전골', '푸짐', '하', '게', '먹', '고', '와서']
['포스팅', '해', '봅니다']
['외관', '사진', '입니다']
['멀리서', '봐도', '한눈', '에', '보이', '는', '외관', '입니다']
['주차', '는', '매장', '뒤쪽', '이나', '매장', '앞', '도로', '시간', '에', '따라', '가능', '에', '하', '시', '면', '됩니다']
['메뉴판', '입니다']
['대표', '메뉴', '인', '만두전골', '을', '시작', '으로']
['바지락', '칼국수', '들깨', '칼국수', '파전', '등', '이', '있', '습니다']
['만두전골', '2', '인', '주문', '해', '봤', '습니다']
['부추', '랑', '양파', '가', '들어간', '소스']
['기본', '찬', '입니다']
['배추김치', '도토리묵', '열무김치']
['콘', '옥수수', '마요네즈', '에', '버무려서', '살찔', '것', '같', '은', '데', '먹', '게', '되', '는', '거']
['고기', '가', '먼저', '나왔', '습니다']
['다른', '전골', '집', '에서', '나오', '는', '고기', '보다', '큼직', '하', '게', '나옵니다']
['두께', '는', '비슷', '했', '고요']
['만두', '는', '1', '인분', '에', '3', '개', '씩', '제공']
['만두', '가', '좀', '적', '다고', '생각', '했', '는데']
['버섯', '이', '종류', '별', '로', '굉장히', '많이', '나옵니다']
['비주얼', '이', '좋', '습니다']
['전골', '에', '버섯', '넣', '으면', '맛', '이', '또', '좋', '잖아요']
['팔', '팔', '끓', '고', '있', '습니다']
['고기', '도', '조금', '씩